# RNNs with custom embeddings

## Importing data

In [1]:
import copy
from itertools import product
from numpy import isnan
import pandas as pd
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext.data as data
import sys
import warnings
warnings.filterwarnings('ignore')

sys.path.append('../data_pipeline/')
import preprocessing as pre
from training import TrainingModule

SEED = 1312
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [2]:
train_data, test_data, val_data, TEXT, LABEL = pre.get_data(
    'train_small.csv', 'val_small.csv', 'test_small.csv')

Connected!


In [3]:
USE_CUDA = torch.cuda.is_available()

TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)

BATCH_SIZE = 5

train_it, test_it, val_it = data.BucketIterator.splits(
    (train_data, test_data, val_data), 
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.alj_text),
    sort_within_batch=True,
    device = torch.device('cuda' if USE_CUDA else 'cpu'))

## RNN Model

In [4]:
class RNN(nn.Module):
    
    def __init__(self, rnn_type, input_size, embedding_size,
                 hidden_size, output_size, num_layers, dropout,
                 bidirectional, padding_idx):
    
        super().__init__()
        self.embedding = nn.Embedding(input_size, embedding_size,
                                      padding_idx=padding_idx)
        
        self.rnn = getattr(nn, rnn_type.upper())\
                          (embedding_size, hidden_size, num_layers,
                           dropout=(dropout if num_layers > 1 else 0),
                           bidirectional=bidirectional)
        
        self.leakyrelu = nn.LeakyReLU()
        self.dropout = nn.Dropout(dropout)
        
        linear_inp = (hidden_size * 2 if bidirectional else hidden_size)
        self.linear = nn.Linear(linear_inp, output_size)
             
    def forward(self, input):
        embed = self.embedding(input)
        rnn_out, hidden = self.rnn(embed)
        rnn_out = rnn_out[-1]
        rnn_out = self.leakyrelu(rnn_out)
        rnn_out = self.dropout(rnn_out)
        linear_out = self.linear(rnn_out)
        return linear_out

## Training models

In [5]:
# Store training results
df = pd.DataFrame(columns=['architecture', 'model_type', 'embeddings',
                           'hidden', 'num_layers', 'dropouts',
                           'bidirectional', 'learning_rate', 'epochs',
                           'dev_acc', 'dev_prec', 'dev_recall',
                           'metric'])

# Model architecture parameters
RNN_TYPES = ['RNN', 'LSTM']
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_SIZES = [32, 64, 128, 256]
HIDDEN_SIZES = [1/3, 2/3]
OUTPUT_SIZE = 1
NUM_LAYERS = [1, 2]
DROPOUTS = [0.5, 0.75]
BIDIRECTIONALS = [False, True]
PADDING_IDX = TEXT.vocab.stoi[TEXT.pad_token]

# Model training hyperparameters
LEARNING_RATE = [0.01, 0.0001]
train_len = 0
train_pos = 0
for batch in train_it:
    train_len += len(batch.decision_binary)
    train_pos += batch.decision_binary.sum().item()
POS_WEIGHT = torch.tensor([(train_len - train_pos) / train_pos])
if USE_CUDA:
    POS_WEIGHT = POS_WEIGHT.cuda()
EPOCHS = 10

# Iterator over various model parameters
param_iter = product (RNN_TYPES, EMBEDDING_SIZES, HIDDEN_SIZES,
                      NUM_LAYERS, DROPOUTS, BIDIRECTIONALS,
                      LEARNING_RATE)

# Magic loop
best_acc = (None, None)
best_rec = (None, None)
best_prec = (None, None)
for i, (rnn_type, embed_size, hidden_size, num_layers, dropout,\
    bidirectional, lr) in enumerate(param_iter):
    print(f'Architecture #{i}\n' + '-' * 20)
    hidden_dim = int(hidden_size * embed_size)
    model = RNN(rnn_type, INPUT_DIM, embed_size, hidden_dim,
                OUTPUT_SIZE, num_layers, dropout, bidirectional,
                PADDING_IDX)
    
    tm = TrainingModule(model, lr, POS_WEIGHT, USE_CUDA, EPOCHS)
    
    best_models = tm.train_model(train_it, val_it)
    
    for metric, best_model in best_models.items():
        row = [i, rnn_type, embed_size, hidden_size, num_layers,
               dropout, bidirectional, lr, EPOCHS,
               best_model.accuracy, best_model.precision,
               best_model.recall, metric]
        df.loc[len(df)] = row
        if best_acc[0] is None or isnan(best_acc[1]) or\
           best_model.accuracy > best_acc[1]:
            best_acc = (copy.deepcopy(best_model.model), best_model.accuracy)
        if best_rec[0] is None or isnan(best_rec[1]) or\
           best_model.recall > best_rec[1]:
            best_rec = (copy.deepcopy(best_model.model), best_model.recall)
        if best_prec[0] is None or isnan(best_prec[1]) or\
           best_model.precision > best_prec[1]:
            best_prec = (copy.deepcopy(best_model.model), best_model.precision)
    
    print('-' * 20 + '\n')


Architecture #0
--------------------
Epoch 0: Dev Accuracy: 0.1525; Dev Precision: 0.1250; Dev Recall: 0.8750; Dev Loss:0.1647
Epoch 1: Dev Accuracy: 0.1864; Dev Precision: 0.1429; Dev Recall: 1.0000; Dev Loss:0.1620
Epoch 2: Dev Accuracy: 0.1864; Dev Precision: 0.1429; Dev Recall: 1.0000; Dev Loss:0.1645
Epoch 3: Dev Accuracy: 0.8644; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.1591
Epoch 4: Dev Accuracy: 0.1695; Dev Precision: 0.1404; Dev Recall: 1.0000; Dev Loss:0.1711
Epoch 5: Dev Accuracy: 0.8136; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.1518
Epoch 6: Dev Accuracy: 0.1695; Dev Precision: 0.1404; Dev Recall: 1.0000; Dev Loss:0.1636
Epoch 7: Dev Accuracy: 0.1695; Dev Precision: 0.1404; Dev Recall: 1.0000; Dev Loss:0.1764
Epoch 8: Dev Accuracy: 0.7458; Dev Precision: 0.1111; Dev Recall: 0.1250; Dev Loss:0.1607
Epoch 9: Dev Accuracy: 0.2712; Dev Precision: 0.1569; Dev Recall: 1.0000; Dev Loss:0.1617
--------------------

Architecture #1
--------------------
Epoch 0

Epoch 6: Dev Accuracy: 0.2373; Dev Precision: 0.1509; Dev Recall: 1.0000; Dev Loss:0.1662
Epoch 7: Dev Accuracy: 0.1525; Dev Precision: 0.1379; Dev Recall: 1.0000; Dev Loss:0.1671
Epoch 8: Dev Accuracy: 0.1356; Dev Precision: 0.1356; Dev Recall: 1.0000; Dev Loss:0.1749
Epoch 9: Dev Accuracy: 0.2203; Dev Precision: 0.1481; Dev Recall: 1.0000; Dev Loss:0.1634
--------------------

Architecture #9
--------------------
Epoch 0: Dev Accuracy: 0.1356; Dev Precision: 0.1356; Dev Recall: 1.0000; Dev Loss:0.1808
Epoch 1: Dev Accuracy: 0.1356; Dev Precision: 0.1356; Dev Recall: 1.0000; Dev Loss:0.1794
Epoch 2: Dev Accuracy: 0.1356; Dev Precision: 0.1356; Dev Recall: 1.0000; Dev Loss:0.1777
Epoch 3: Dev Accuracy: 0.1356; Dev Precision: 0.1356; Dev Recall: 1.0000; Dev Loss:0.1761
Epoch 4: Dev Accuracy: 0.1356; Dev Precision: 0.1356; Dev Recall: 1.0000; Dev Loss:0.1749
Epoch 5: Dev Accuracy: 0.1356; Dev Precision: 0.1356; Dev Recall: 1.0000; Dev Loss:0.1744
Epoch 6: Dev Accuracy: 0.1356; Dev Precis

Epoch 2: Dev Accuracy: 0.8644; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.1512
Epoch 3: Dev Accuracy: 0.8644; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.1516
Epoch 4: Dev Accuracy: 0.8644; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.1525
Epoch 5: Dev Accuracy: 0.8644; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.1532
Epoch 6: Dev Accuracy: 0.8644; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.1538
Epoch 7: Dev Accuracy: 0.8644; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.1544
Epoch 8: Dev Accuracy: 0.8644; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.1546
Epoch 9: Dev Accuracy: 0.8644; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.1550
--------------------

Architecture #18
--------------------
Epoch 0: Dev Accuracy: 0.2034; Dev Precision: 0.1455; Dev Recall: 1.0000; Dev Loss:0.1678
Epoch 1: Dev Accuracy: 0.2542; Dev Precision: 0.1538; Dev Recall: 1.0000; Dev Loss:0.1642
Epoch 2: Dev Accuracy: 0.7458; Dev Precision: 0.0000; Dev Recall

Epoch 8: Dev Accuracy: 0.1695; Dev Precision: 0.1404; Dev Recall: 1.0000; Dev Loss:0.1632
Epoch 9: Dev Accuracy: 0.1695; Dev Precision: 0.1404; Dev Recall: 1.0000; Dev Loss:0.1631
--------------------

Architecture #26
--------------------
Epoch 0: Dev Accuracy: 0.7627; Dev Precision: 0.1250; Dev Recall: 0.1250; Dev Loss:0.1547
Epoch 1: Dev Accuracy: 0.8644; Dev Precision: 0.5000; Dev Recall: 0.1250; Dev Loss:0.1592
Epoch 2: Dev Accuracy: 0.8475; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.1557
Epoch 3: Dev Accuracy: 0.1356; Dev Precision: 0.1356; Dev Recall: 1.0000; Dev Loss:0.1806
Epoch 4: Dev Accuracy: 0.2712; Dev Precision: 0.1569; Dev Recall: 1.0000; Dev Loss:0.1620
Epoch 5: Dev Accuracy: 0.1356; Dev Precision: 0.1356; Dev Recall: 1.0000; Dev Loss:0.1735
Epoch 6: Dev Accuracy: 0.1356; Dev Precision: 0.1356; Dev Recall: 1.0000; Dev Loss:0.1664
Epoch 7: Dev Accuracy: 0.1356; Dev Precision: 0.1356; Dev Recall: 1.0000; Dev Loss:0.1781
Epoch 8: Dev Accuracy: 0.8136; Dev Preci

Epoch 4: Dev Accuracy: 0.1695; Dev Precision: 0.1404; Dev Recall: 1.0000; Dev Loss:0.1773
Epoch 5: Dev Accuracy: 0.5763; Dev Precision: 0.0526; Dev Recall: 0.1250; Dev Loss:0.1697
Epoch 6: Dev Accuracy: 0.7119; Dev Precision: 0.0909; Dev Recall: 0.1250; Dev Loss:0.1537
Epoch 7: Dev Accuracy: 0.6610; Dev Precision: 0.1250; Dev Recall: 0.2500; Dev Loss:0.1659
Epoch 8: Dev Accuracy: 0.1356; Dev Precision: 0.1228; Dev Recall: 0.8750; Dev Loss:0.1953
Epoch 9: Dev Accuracy: 0.5932; Dev Precision: 0.1364; Dev Recall: 0.3750; Dev Loss:0.1689
--------------------

Architecture #35
--------------------
Epoch 0: Dev Accuracy: 0.1356; Dev Precision: 0.1356; Dev Recall: 1.0000; Dev Loss:0.1712
Epoch 1: Dev Accuracy: 0.1356; Dev Precision: 0.1356; Dev Recall: 1.0000; Dev Loss:0.1694
Epoch 2: Dev Accuracy: 0.1356; Dev Precision: 0.1356; Dev Recall: 1.0000; Dev Loss:0.1683
Epoch 3: Dev Accuracy: 0.1356; Dev Precision: 0.1356; Dev Recall: 1.0000; Dev Loss:0.1676
Epoch 4: Dev Accuracy: 0.1356; Dev Preci

...

Epoch 8: Dev Accuracy: 0.7797; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.1612
Epoch 9: Dev Accuracy: 0.7797; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.1612
--------------------

Architecture #224
--------------------
Epoch 0: Dev Accuracy: 0.2034; Dev Precision: 0.1455; Dev Recall: 1.0000; Dev Loss:0.1636
Epoch 1: Dev Accuracy: 0.1356; Dev Precision: 0.1356; Dev Recall: 1.0000; Dev Loss:0.1764
Epoch 2: Dev Accuracy: 0.1864; Dev Precision: 0.1429; Dev Recall: 1.0000; Dev Loss:0.1720
Epoch 3: Dev Accuracy: 0.2373; Dev Precision: 0.1509; Dev Recall: 1.0000; Dev Loss:0.1594
Epoch 4: Dev Accuracy: 0.7627; Dev Precision: 0.1250; Dev Recall: 0.1250; Dev Loss:0.1606
Epoch 5: Dev Accuracy: 0.7797; Dev Precision: 0.2222; Dev Recall: 0.2500; Dev Loss:0.1678
Epoch 6: Dev Accuracy: 0.7797; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.1440
Epoch 7: Dev Accuracy: 0.6441; Dev Precision: 0.1579; Dev Recall: 0.3750; Dev Loss:0.1604
Epoch 8: Dev Accuracy: 0.5085; Dev Prec

Epoch 3: Dev Accuracy: 0.1356; Dev Precision: 0.1356; Dev Recall: 1.0000; Dev Loss:0.1735
Epoch 4: Dev Accuracy: 0.8305; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.1574
Epoch 5: Dev Accuracy: 0.1356; Dev Precision: 0.1356; Dev Recall: 1.0000; Dev Loss:0.1777
Epoch 6: Dev Accuracy: 0.7458; Dev Precision: 0.1111; Dev Recall: 0.1250; Dev Loss:0.1683
Epoch 7: Dev Accuracy: 0.2373; Dev Precision: 0.1373; Dev Recall: 0.8750; Dev Loss:0.1819
Epoch 8: Dev Accuracy: 0.6949; Dev Precision: 0.0833; Dev Recall: 0.1250; Dev Loss:0.1852
Epoch 9: Dev Accuracy: 0.7627; Dev Precision: 0.2500; Dev Recall: 0.3750; Dev Loss:0.1719
--------------------

Architecture #233
--------------------
Epoch 0: Dev Accuracy: 0.8644; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.1572
Epoch 1: Dev Accuracy: 0.8136; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.1582
Epoch 2: Dev Accuracy: 0.7797; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.1587
Epoch 3: Dev Accuracy: 0.7797; Dev Precisi

Epoch 9: Dev Accuracy: 0.1525; Dev Precision: 0.1379; Dev Recall: 1.0000; Dev Loss:0.2080
--------------------

Architecture #241
--------------------
Epoch 0: Dev Accuracy: 0.1864; Dev Precision: 0.1296; Dev Recall: 0.8750; Dev Loss:0.1660
Epoch 1: Dev Accuracy: 0.1864; Dev Precision: 0.1296; Dev Recall: 0.8750; Dev Loss:0.1647
Epoch 2: Dev Accuracy: 0.1864; Dev Precision: 0.1296; Dev Recall: 0.8750; Dev Loss:0.1643
Epoch 3: Dev Accuracy: 0.1864; Dev Precision: 0.1296; Dev Recall: 0.8750; Dev Loss:0.1658
Epoch 4: Dev Accuracy: 0.1695; Dev Precision: 0.1273; Dev Recall: 0.8750; Dev Loss:0.1651
Epoch 5: Dev Accuracy: 0.7288; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.1642
Epoch 6: Dev Accuracy: 0.7119; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.1639
Epoch 7: Dev Accuracy: 0.7627; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.1619
Epoch 8: Dev Accuracy: 0.7119; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.1636
Epoch 9: Dev Accuracy: 0.7458; Dev Prec

Epoch 4: Dev Accuracy: 0.7797; Dev Precision: 0.1429; Dev Recall: 0.1250; Dev Loss:0.1606
Epoch 5: Dev Accuracy: 0.8136; Dev Precision: 0.2000; Dev Recall: 0.1250; Dev Loss:0.1592
Epoch 6: Dev Accuracy: 0.7458; Dev Precision: 0.1111; Dev Recall: 0.1250; Dev Loss:0.1601
Epoch 7: Dev Accuracy: 0.7119; Dev Precision: 0.0909; Dev Recall: 0.1250; Dev Loss:0.1613
Epoch 8: Dev Accuracy: 0.7627; Dev Precision: 0.2500; Dev Recall: 0.3750; Dev Loss:0.1506
Epoch 9: Dev Accuracy: 0.6610; Dev Precision: 0.2000; Dev Recall: 0.5000; Dev Loss:0.1613
--------------------

Architecture #250
--------------------


RuntimeError: CUDA out of memory. Tried to allocate 2.49 GiB (GPU 0; 14.76 GiB total capacity; 7.48 GiB already allocated; 27.44 MiB free; 9.82 GiB reserved in total by PyTorch) (malloc at /pytorch/c10/cuda/CUDACachingAllocator.cpp:289)
frame #0: c10::Error::Error(c10::SourceLocation, std::string const&) + 0x46 (0x7ff7154bc536 in /usr/local/lib/python3.6/dist-packages/torch/lib/libc10.so)
frame #1: <unknown function> + 0x1cf1e (0x7ff715705f1e in /usr/local/lib/python3.6/dist-packages/torch/lib/libc10_cuda.so)
frame #2: <unknown function> + 0x1df9e (0x7ff715706f9e in /usr/local/lib/python3.6/dist-packages/torch/lib/libc10_cuda.so)
frame #3: at::native::empty_cuda(c10::ArrayRef<long>, c10::TensorOptions const&, c10::optional<c10::MemoryFormat>) + 0x135 (0x7ff71829a535 in /usr/local/lib/python3.6/dist-packages/torch/lib/libtorch_cuda.so)
frame #4: <unknown function> + 0xf7a66b (0x7ff71689266b in /usr/local/lib/python3.6/dist-packages/torch/lib/libtorch_cuda.so)
frame #5: <unknown function

As a result of the the GPU we were using running out of memory, we we were unable to train the final six model specifications. The previous two-hundred and fifty model specifications, however, were preserved and results are reported for them. In particular, the models that we were unable to train were all bidirectional, stacked (two-layer) LSTM networks. Given the time-constraints on the project and the over five-hundred other models that we successfully trained across this and other architectures, we decided to set these failed network architectures aside and focus on analyzing the output we successfully obtained.

## Save results of model training

In [6]:
SAVE_PREFIX = '../results/RNNCustom_'
df.to_csv(f'{SAVE_PREFIX}models.csv')
torch.save(best_acc[0], f'{SAVE_PREFIX}best_acc.pt')
torch.save(best_rec[0], f'{SAVE_PREFIX}best_rec.pt')
torch.save(best_prec[0], f'{SAVE_PREFIX}best_prec.pt')